# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现线性回归模型。

## 1. 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 2. 读取数据集

调用框架中现有的 API 来读取数据：

In [3]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个 Pytorch 数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [4]:
next(iter(data_iter))

[tensor([[-1.1859, -1.4592],
         [ 0.9004,  0.5667],
         [ 0.6350, -0.4150],
         [-0.6325,  1.6290],
         [ 0.8067,  1.5645],
         [-0.4751, -0.6391],
         [-1.1662, -0.5351],
         [ 0.6765,  0.4735],
         [-1.1674,  0.4409],
         [ 0.2446, -0.9875]]),
 tensor([[ 6.7860],
         [ 4.0750],
         [ 6.8753],
         [-2.6114],
         [ 0.5012],
         [ 5.4140],
         [ 3.6883],
         [ 3.9435],
         [ 0.3636],
         [ 8.0390]])]

## 3. 定义模型

对于标准的深度学习模型，我们可以使用 Pytorch 框架预先定义好的层。这使我们只需关注使用那些层来构造模型，而不必关注层的实现细节。

定义一个模型变量 net，它是一个 Sequential 类的实例。Sequential 类将多个层串联在一起，当给定输入数据时，Sequential 实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，以此类推。在下面的例子中，我们的模型只包含一个层，因此实际上不需要 Sequential，但是因为以后几乎所有模型都是多层的，在这里使用 Sequential 会让你熟悉“标准的流水线”。

回顾单层网络架构，这一单层被称为全连接层（fully-connected layer），因为它的每个输入都通过矩阵-向量乘法得到它的每个输出。

在 Pytorch 中，全连接层在 Linear 类中定义，值得注意的是，我们将两个参数传递到 `nn.Linear` 中，第一个参数指定输入特征的形状，第二个参数指定输出特征的形状。

In [5]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

## 4. 初始化模型参数

通过 `net[0]` 选择网络中的第一个层，然后使用 `weight.data` 和 `bias.data` 方法访问参数：

In [6]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 5. 定义损失函数

计算均方误差使用的是 MSELoss 类，也称为平方 $L_2$ 范数。默认情况下，它返回所有样本损失的平均值：

In [7]:
loss = nn.MSELoss()

## 6. 定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具，Pytorch 在 optim 模块中实现了该算法的许多变种。当我们实例化一个 `SGD` 实例时，需要指定要优化的参数（可通过 `net.parameters()` 从我们的模型中获得）以及优化算法所需的超参数字典（小批量随机梯度下降只需设置 `lr` 值，这里设置为 0.03）：

In [8]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 7. 训练

训练过程代码与我们从零开始实现时所做的非常相似：
- 通过调用 net(X) 生成预测并计算损失 l（前向传播）
- 通过进行反向传播来计算梯度
- 通过调用优化器来更新模型参数

In [9]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)  # net本身自带了weight和bias
        trainer.zero_grad()  # 清零梯度
        l.backward()  ## pytorch帮我们做了sum()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch: {epoch+1}, loss: {l: f}')

epoch: 1, loss:  0.000286
epoch: 2, loss:  0.000102
epoch: 3, loss:  0.000103


比较真实参数和通过训练学到的参数来评估训练的成功程度：

In [10]:
w = net[0].weight.data
b = net[0].bias.data
print(f'w 的估计误差：{true_w - w.reshape(true_w.shape)}')
print(f'b 的估计误差：{true_b - b}')

w 的估计误差：tensor([-0.0006,  0.0005])
b 的估计误差：tensor([0.0002])


## 练习：

1. 如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率？
   $$l\div batchsize$$
   
3. 如何访问线性回归的梯度？

In [14]:
net[0].weight.grad, net[0].bias.grad

(tensor([[ 0.0029, -0.0190]]), tensor([0.0151]))

## Q & A

1. 推荐一些对学生到比较便宜的平台。

   Google colab

2. 为什么使用平方损失而不是用绝对差值呢？

   其实区别不大，最早大家使用平方损失是因为绝对差值在 0 点处是不可导的。

3. 损失为什么要求平均？

   使用随机梯度下降的话，如果不除以n，相当于计算出的梯度比较大。除以n的好处是，不管批量大小，损失受批量大小的影响不大，这样以来学习率比较好调整。损失不除以n，只需要将学习率除以n就行了。

4. 线性回归的损失函数是不是通常是 MSE ？

   一般是这样的。

5. 不管是梯度下降还是随机梯度下降，如何快速找到合适的学习率？

    - 找到一个对学习率不那么敏感的优化算法，例如 Adam 这种比较 smooth 的梯度下降
    - 合理初始化参数，使得学习率为 0.01 或 0.1 就差不多了，后面会讲数据稳定性
    - 后面会讲到调整学习率的办法

  
6. 批量大小对模型优化的影响？

   其实 batch_size 过小是好的，太大了不行，批量越小其实对收敛越好，小批量会对模型带来噪音，这是一件好事情，因为现在的深度学习网络都是很复杂的，对各种噪音的容忍度越好，说明模型的泛化性越好。

7. 过拟合和欠拟合如何调整 batch_size 和 lr ？

   学习率和批量大小理论上不会影响模型最后的收敛效果，实际上有一定的影响，但不需要特别纠结，批量大小只要不是特别大，多一点时间模型最后还是可以收敛的。

8. 随机梯度下降的随机是指？

   每次在样本中随机采样一定批量的样本。

9. 在深度学习中，设置损失函数时，需要考虑正则吗？

   需要的，但是损失函数和正则一般是分开的。

10. 为什么机器学习的优化算法都采取梯度下降（一阶导算法），而不采用牛顿法（二阶导算法），收敛速度更快，一般能算出一阶导，二阶导应该也能算。

    首先，“一般能算出一阶导，二阶导应该也能算”这句话不一定是对的，二阶导并不是总能算，是很贵的，因为一阶梯度是个向量(eg.100x1)，而二阶梯度就是一个矩阵（100x100）了。第二，为什么不用牛顿法？神经网络中用到的一个是统计模型，一个是优化模型，这两个模型其实都是错的，一般来说，实际上机器学习是求不到最优解的，我们把统计模型的最优解求出来其实意义并不大，因为它是错的。真实的模型很复杂，牛顿法求出来的可能并不是最优解而是次优解，不一定比随机梯度下降求出来的更好。总之，牛顿法：1. 计算比较复杂；2. 收敛不一定快，就算快，结果不一定最好；所以相比较而言选择更简单的梯度下降。

11. detach() 的作用？

    不要自动求梯度了。

12. 本例中 data_iter 的写法，每次把所有数据 load 进内存，内存会爆掉，有好的解决办法吗？

    实际上，不会把所有数据 load 进内存，或者往前读（pre_fetch）几个批量大小，但是教程中的例子数据不会特别大，所以 load 进内存没有太大的问题。

13. 如果样本数量不是 batch_size 的整数倍？

    - (a).最后一个批次拿剩下的小一点的样本；
    - (b).丢掉最后一个批次；
    - (c).从下一个 epoch 中拿样本补齐。
   
14. 学习率不需要做衰减吗？

    理论上 SGD 要收敛的话，需要不断的减小学习率；但实际上有很多其他办法，这里先不讨论。

15. 本质上我们为什么要用 SGD ？

    因为实际上大部分 loss 太复杂，推导不出导数为 0 的解，只能逐个 batch 去逼近。只有线性回归有显示解，其他模型都没有显示解。有显示解就没必要使用深度学习了。

16. 定义网络层一定要手动设置初始值吗？

    不需要，有默认的初始值。

17. 实际中 loss 输出 nan 和 inf ？

    后面会讲到数据稳定性。